In [1]:
!git clone https://github.com/microsoft/table-transformer
!cd table-transformer

fatal: destination path 'table-transformer' already exists and is not an empty directory.


In [2]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [3]:
!cd table-transformer

In [4]:
!pip install pdf2image

In [5]:
!pip install transformers

In [6]:
!pip install timm

In [7]:
!pip


Usage:   
  pip <command> [options]

Commands:
  install                     Install packages.
  download                    Download packages.
  uninstall                   Uninstall packages.
  freeze                      Output installed packages in requirements format.
  inspect                     Inspect the python environment.
  list                        List installed packages.
  show                        Show information about installed packages.
  check                       Verify installed packages have compatible dependencies.
  config                      Manage local and global configuration.
  search                      Search PyPI for packages.
  cache                       Inspect and manage pip's wheel cache.
  index                       Inspect information available from package indexes.
  wheel                       Build wheels from your requirements.
  hash                        Compute hashes of package archives.
  completion                  A helper co

In [8]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install --upgrade torch pdf2image pandas


Note: you may need to restart the kernel to use updated packages.


In [12]:
import torch
from transformers import TableTransformerForObjectDetection, AutoTokenizer
from pdf2image import convert_from_path
import pandas as pd

# Load the pre-trained TATR model and tokenizer
model_name = "microsoft/table-transformer-detection"
model = TableTransformerForObjectDetection.from_pretrained(model_name)



Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
# Convert PDF pages to images
def convert_pdf_to_images(pdf_path, dpi=300):
    images = convert_from_path(pdf_path, dpi=dpi)
    return images


In [14]:
from PIL import Image
import torch

# Function to run table detection using the TATR model
def run_table_detection(image, model):
    inputs = tokenizer(images=image, return_tensors="pt")

    # Forward pass through the model
    outputs = model(**inputs)

    # Process model outputs
    table_boxes = outputs.pred_boxes
    return table_boxes

# Function to extract tables from images
def extract_tables_from_images(images, model):
    table_data = []
    for image in images:
        boxes = run_table_detection(image, model)
        table_data.append(boxes)
    return table_data


In [15]:
# Function to save the extracted tables to CSV
def save_tables_to_csv(tables, output_path):
    for i, table in enumerate(tables):
        table_df = pd.DataFrame(table)  # Convert table data to DataFrame
        csv_filename = f"{output_path}/table_{i+1}.csv"
        table_df.to_csv(csv_filename, index=False)
        print(f"Table {i+1} saved to {csv_filename}")

# Function to save the extracted tables to Excel
def save_tables_to_excel(tables, output_path):
    writer = pd.ExcelWriter(f"{output_path}/tables_output.xlsx")
    for i, table in enumerate(tables):
        table_df = pd.DataFrame(table)
        table_df.to_excel(writer, sheet_name=f'Table_{i+1}', index=False)
    writer.save()
    print("Tables saved to Excel file.")


In [16]:
pip install poppler-utils

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [22]:

#from pdf2image import convert_from_path

# # Specify the path to poppler if needed (only if it’s not in PATH)
#poppler_path = r'C:\poppler\poppler-24.08.0\Library\bin'  # Replace with the actual path on Windows


In [ ]:
import torch
from transformers import TableTransformerForObjectDetection, DetrImageProcessor
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
import os

# Function to enlarge the bounding box
def enlarge_box(box, scale_factor=1.4):
    # Center of the box
    x_center = (box[0] + box[2]) / 2
    y_center = (box[1] + box[3]) / 2
    width = (box[2] - box[0]) * scale_factor
    height = (box[3] - box[1]) * scale_factor

    # New coordinates
    x_min = x_center - width / 2
    y_min = y_center - height / 2
    x_max = x_center + width / 2
    y_max = y_center + height / 2

    return [x_min, y_min, x_max, y_max]

# Function to run table detection using the TATR model
def run_table_detection(image, model, processor):
    inputs = processor(images=image, return_tensors="pt")

    # Forward pass through the model
    outputs = model(**inputs)

    # Get predicted boxes
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

    return results

# Function to extract tables from a PDF
def extract_tables_from_pdf(pdf_path, output_dir, model, processor, scale_factor=1.2):
    # Convert PDF pages to images
    images = convert_from_path(pdf_path, poppler_path=r"C:\poppler\poppler-24.08.0\Library\bin")

    for page_number, image in enumerate(images):
        # Run table detection on each page
        results = run_table_detection(image, model, processor)

        # Plot detected tables
        fig, ax = plt.subplots(1, figsize=(16, 10))
        ax.imshow(image)

        for box in results['boxes'].detach().numpy():  # Detach tensor before converting to numpy
            # Enlarge the bounding box by the scale factor
            enlarged_box = enlarge_box(box, scale_factor)

            # Add rectangle to the plot for the detected table
            ax.add_patch(plt.Rectangle((enlarged_box[0], enlarged_box[1]), enlarged_box[2] - enlarged_box[0], enlarged_box[3] - enlarged_box[1],
                                       fill=False, color="red", linewidth=3))

        plt.axis('off')
        plt.savefig(os.path.join(output_dir, f"page_{page_number+1}_tables.png"))
        plt.close()

# Load the Table Transformer model and image processor
model_name = "microsoft/table-transformer-detection"
model = TableTransformerForObjectDetection.from_pretrained(model_name)
processor = DetrImageProcessor.from_pretrained(model_name)

# Replace pdf_path with your actual file path on Windows
# pdf_path = r"D:\InternshipProject\projects\content\sample_data\1.pdf"  # Use raw string (r"") to avoid escape issues
output_dir = r"C:\Users\kandi\Desktop\projects\projects\content\poorna\enlargeoutput_table"  # Also update the output directory if needed

# Create output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Extract tables and save them
extract_tables_from_pdf(pdf_path, output_dir, model, processor, scale_factor=1.2)

print(f"Table extraction completed! Enlarged tables are saved in {output_dir}")


Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Table extraction completed! Enlarged tables are saved in D:\InternshipProject\projects\content\poorna\enlargeoutput_table


In [25]:
!pip install pytesseract

In [26]:
#extract tables image from full images-output

enlarge tables images-enlargeoutput

In [32]:
!pip install openpyxl

In [ ]:
import os
import torch
from transformers import TableTransformerForObjectDetection, DetrImageProcessor
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import pandas as pd
import openpyxl


# Function to enlarge the bounding box
def enlarge_box(box, scale_factor=1.2):
    # Center of the box
    x_center = (box[0] + box[2]) / 2
    y_center = (box[1] + box[3]) / 2
    width = (box[2] - box[0]) * scale_factor
    height = (box[3] - box[1]) * scale_factor

    # New coordinates
    x_min = x_center - width / 2
    y_min = y_center - height / 2
    x_max = x_center + width / 2
    y_max = y_center + height / 2

    return [x_min, y_min, x_max, y_max]

# Function to run table detection using the TATR model
def run_table_detection(image, model, processor):
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    # Get predicted boxes
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]
    return results

# Function to crop detected table regions, enlarge, and save as images
def crop_and_save_tables(image, results, output_dir, page_number, scale_factor=1.2):
    tables_data = []
    for i, box in enumerate(results['boxes'].detach().numpy()):
        # Enlarge the detected table box
        enlarged_box = enlarge_box(box, scale_factor)

        # Crop the enlarged table region
        cropped_image = image.crop((enlarged_box[0], enlarged_box[1], enlarged_box[2], enlarged_box[3]))
        cropped_image_path = os.path.join(output_dir, f"page_{page_number+1}_table_{i+1}.png")
        cropped_image.save(cropped_image_path)

        # Convert cropped table image to text using OCR (Tesseract)
        table_text = pytesseract.image_to_string(cropped_image, config="--psm 6")
        tables_data.append(table_text)
    return tables_data

# Function to extract tables from a PDF and save them as CSV or Excel
def extract_tables_to_excel(pdf_path, output_dir, model, processor, scale_factor=1.2):
    images = convert_from_path(pdf_path, poppler_path=r"C:\poppler\poppler-24.08.0\Library\bin")
    all_tables = []

    for page_number, image in enumerate(images):
        results = run_table_detection(image, model, processor)
        tables_data = crop_and_save_tables(image, results, output_dir, page_number, scale_factor)

        for table_text in tables_data:
            table_rows = [row.split() for row in table_text.splitlines() if row.strip()]
            all_tables.extend(table_rows)

    df = pd.DataFrame(all_tables)
    output_file_path = os.path.join(output_dir, "extracted_tables.xlsx")
    df.to_excel(output_file_path, index=False)
    return output_file_path

# Load the Table Transformer model and image processor
model_name = "microsoft/table-transformer-detection"
model = TableTransformerForObjectDetection.from_pretrained(model_name)
processor = DetrImageProcessor.from_pretrained(model_name)

# Replace pdf_path with your actual file path on Windows
# pdf_path = r"D:\InternshipProject\projects\content\sample_data\1.pdf"  # Use raw string (r"") to avoid escape issues
output_dir = r"C:\Users\kandi\Desktop\projects\projects\content\poorna\enlargeoutput"  # Also update the output directory if needed

# Create output directory if it does not exist
if not os.makedirs(output_dir, exist_ok=True):
    print(f"Directory {output_dir} created.")

# # Install tesseract-ocr if not installed
# !apt-get install -y tesseract-ocr
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


# # Extract tables and save them as Excel
output_file = extract_tables_to_excel(pdf_path, output_dir, model, processor, scale_factor=1.2)

# # Provide the generated Excel file to the user
# output_file


Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Directory D:\InternshipProject\projects\content\poorna\enlargeoutput created.


image to csv-enlargecsv_folder
NEXT STAGES: 
after careful observation we came to know that csv tables extarct is not correct so to increase effeciency i am using extract table api to get csv from images
extraction of tables from image

In [34]:
!git clone https://github.com/ExtractTable/ExtractTable-py.git

Cloning into 'ExtractTable-py'...


In [35]:
!pip install ExtractTable

use ur own api :https://extracttable.com/signup/trial.html

In [ ]:
import os
from ExtractTable import ExtractTable
import pandas as pd

# Initialize ExtractTable API client with your API key
api_key = "k97NYypymJdUCrwc30GeHgjyiumiirC6aiVa9WK7"  # Replace this with your actual API key
et_sess = ExtractTable(api_key=api_key)

# Function to extract tables from images and save them as CSV
def extract_tables_from_images(image_folder_path, output_folder):
    # Get list of all image file paths in the folder
    image_paths = [os.path.join(image_folder_path, file) for file in os.listdir(image_folder_path) if file.endswith(('.png', '.jpg', '.jpeg'))]

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_path in image_paths:
        try:
            print(f"Processing image: {image_path}")

            # Extract tables from the image using ExtractTable API
            table_data = et_sess.process_file(image_path)

            if table_data:
                for i, table in enumerate(table_data):
                    # Convert the table data to a DataFrame
                    df = pd.DataFrame(table)

                    # Define output file path
                    output_csv_path = os.path.join(output_folder, f"table_{os.path.basename(image_path)}_part_{i+1}.csv")

                    # Save the DataFrame as a CSV file
                    df.to_csv(output_csv_path, index=False)
                    print(f"Table saved to: {output_csv_path}")
            else:
                print(f"No table detected in {image_path}")
        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")

# Paths
image_folder_path = r"C:\Users\kandi\Desktop\projects\projects\content\poorna\enlargeoutput"  # Update this with the actual folder path containing images
output_folder = r"C:\Users\kandi\Desktop\projects\projects\content\poorna\enlargeoutputcs"  # Update this with the folder where you want to save CSVs

# Extract tables from images in the folder and save as CSV
extract_tables_from_images(image_folder_path, output_folder)


Processing image: D:\InternshipProject\projects\content\poorna\enlargeoutput\page_1_table_1.png
Table saved to: D:\InternshipProject\projects\content\poorna\enlargeoutputcs\table_page_1_table_1.png_part_1.csv
Processing image: D:\InternshipProject\projects\content\poorna\enlargeoutput\page_3_table_1.png
Table saved to: D:\InternshipProject\projects\content\poorna\enlargeoutputcs\table_page_3_table_1.png_part_1.csv
Processing image: D:\InternshipProject\projects\content\poorna\enlargeoutput\page_4_table_1.png
Table saved to: D:\InternshipProject\projects\content\poorna\enlargeoutputcs\table_page_4_table_1.png_part_1.csv
Processing image: D:\InternshipProject\projects\content\poorna\enlargeoutput\page_4_table_2.png
Table saved to: D:\InternshipProject\projects\content\poorna\enlargeoutputcs\table_page_4_table_2.png_part_1.csv
Processing image: D:\InternshipProject\projects\content\poorna\enlargeoutput\page_5_table_1.png
Table saved to: D:\InternshipProject\projects\content\poorna\enlarge

removing tables if more than 80 % conatins text

In [ ]:
import os
import pandas as pd
import shutil

# Paths
input_folder = r'C:\Users\kandi\Desktop\projects\projects\content\poorna\enlargeoutputcs'  # Folder containing CSV files
output_folder = r'C:\Users\kandi\Desktop\projects\projects\content\poorna\tables'  # Folder to move valid CSV files
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to calculate the percentage of non-digit cells
def percentage_non_digit_cells(df):
    total_cells = df.size
    non_digit_cells = df.applymap(lambda x: not str(x).replace('.', '', 1).isdigit()).sum().sum()
    return (non_digit_cells / total_cells) * 100

# Iterate through all CSV files in the folder
for file_name in os.listdir(input_folder):
    if file_name.endswith('.csv'):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name}")

        try:
            # Load the CSV to check its contents
            df = pd.read_csv(file_path)

            # Calculate the percentage of non-digit cells
            non_digit_percentage = percentage_non_digit_cells(df)
            print(f"Non-digit percentage for {file_name}: {non_digit_percentage:.2f}%")

            # Check if the percentage of non-digit cells is less than or equal to 15%
            if non_digit_percentage <= 80:
                # Move the file to the output folder if it's valid
                shutil.copy(file_path, os.path.join(output_folder, file_name))
                print(f"Moved {file_name} to {output_folder}")
            else:
                # Delete the file if more than 15% of cells are non-digit

                print(f"Deleted {file_name}: More than 15% of cells are non-digit.")

        except Exception as e:
            print(f"Error processing {file_name}: {e}")

print("Processing complete.")


Processing table_page_1_table_1.png_part_1.csv
Non-digit percentage for table_page_1_table_1.png_part_1.csv: 100.00%
Deleted table_page_1_table_1.png_part_1.csv: More than 15% of cells are non-digit.
Processing table_page_3_table_1.png_part_1.csv
Non-digit percentage for table_page_3_table_1.png_part_1.csv: 21.90%
Moved table_page_3_table_1.png_part_1.csv to D:\InternshipProject\projects\content\poorna\tables
Processing table_page_4_table_1.png_part_1.csv
Non-digit percentage for table_page_4_table_1.png_part_1.csv: 37.00%
Moved table_page_4_table_1.png_part_1.csv to D:\InternshipProject\projects\content\poorna\tables
Processing table_page_4_table_2.png_part_1.csv
Non-digit percentage for table_page_4_table_2.png_part_1.csv: 60.00%
Moved table_page_4_table_2.png_part_1.csv to D:\InternshipProject\projects\content\poorna\tables
Processing table_page_5_table_1.png_part_1.csv
Non-digit percentage for table_page_5_table_1.png_part_1.csv: 36.36%
Moved table_page_5_table_1.png_part_1.csv to 

C:\Users\USER\AppData\Local\Temp\ipykernel_5948\3605188218.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  non_digit_cells = df.applymap(lambda x: not str(x).replace('.', '', 1).isdigit()).sum().sum()
C:\Users\USER\AppData\Local\Temp\ipykernel_5948\3605188218.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  non_digit_cells = df.applymap(lambda x: not str(x).replace('.', '', 1).isdigit()).sum().sum()
C:\Users\USER\AppData\Local\Temp\ipykernel_5948\3605188218.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  non_digit_cells = df.applymap(lambda x: not str(x).replace('.', '', 1).isdigit()).sum().sum()
C:\Users\USER\AppData\Local\Temp\ipykernel_5948\3605188218.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  non_digit_cells = df.applymap(lambda x: not str(x).replace('.', '', 1).isdigit()).sum().sum()
C:\Users\USER\AppData\Lo

adding sample data header

In [ ]:
import os
import pandas as pd

# Path to the folder containing CSV files
csv_folder_path = r"C:\Users\kandi\Desktop\projects\projects\content\poorna\tables"
output_folder_path = r"C:\Users\kandi\Desktop\projects\projects\content\poorna\tableheaders"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Loop through all CSV files in the folder
for file_name in os.listdir(csv_folder_path):
    if file_name.endswith(".csv"):
        file_path = os.path.join(csv_folder_path, file_name)

        # Read the CSV file, skipping the first row (header row in original)
        df = pd.read_csv(file_path, header=None, skiprows=1)

        # Promote the first row to be the header
        df.columns = df.iloc[0]
        df = df[1:]  # Remove the row that became the header

        # Add "Sample" as the first header
        df.columns = ['Sample'] + df.columns.tolist()[1:]

        # Save the updated file to the new folder
        output_file_path = os.path.join(output_folder_path, file_name)
        df.to_csv(output_file_path, index=False)

# List the processed files
processed_files = os.listdir(output_folder_path)
print("Processed files:", processed_files)


Processed files: ['table_page_3_table_1.png_part_1.csv', 'table_page_4_table_1.png_part_1.csv', 'table_page_4_table_2.png_part_1.csv', 'table_page_5_table_1.png_part_1.csv']


getting final data: getting all correct csv tables according to pdf

In [ ]:
import pandas as pd
import os

# Specify the directory containing the original CSV files and the new directory for modified files
input_directory = r"C:\Users\kandi\Desktop\projects\projects\content\poorna\tableheaders"
output_directory =r"C:\Users\kandi\Desktop\projects\projects\content\poorna\finaltable"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate through each file in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.csv'):
        # Load the CSV file
        file_path = os.path.join(input_directory, filename)
        csv_data = pd.read_csv(file_path)

        # Check if 'Sample' column has the 'Casein Red Lentil' entry and proceed with modifications
        if 'Casein Red Lentil' in csv_data['Sample'].values:
            # Replace "Casein Red Lentil" with "Casein"
            csv_data['Sample'] = csv_data['Sample'].replace('Casein Red Lentil', 'Casein')
            # Find the index of the modified row
            index = csv_data[csv_data['Sample'] == 'Casein'].index[0]
            # Insert "Red Lentil" in the next row, clear other cells if the row exists
            if index + 1 < len(csv_data):
                csv_data.loc[index + 1, 'Sample'] = 'Red Lentil'
                csv_data.loc[index + 1, csv_data.columns != 'Sample'] = ''
            else:
                # Add a new row if there is no next row
                new_row = pd.Series({'Sample': 'Red Lentil'})
                csv_data = csv_data.append(new_row, ignore_index=True).fillna('')

        # Save the modified CSV file to the new directory
        output_file_path = os.path.join(output_directory, filename)
        csv_data.to_csv(output_file_path, index=False)

print("All files have been processed and saved to the new directory.")


All files have been processed and saved to the new directory.


C:\Users\USER\AppData\Local\Temp\ipykernel_5948\2548881133.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  csv_data.loc[index + 1, csv_data.columns != 'Sample'] = ''
C:\Users\USER\AppData\Local\Temp\ipykernel_5948\2548881133.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  csv_data.loc[index + 1, csv_data.columns != 'Sample'] = ''
C:\Users\USER\AppData\Local\Temp\ipykernel_5948\2548881133.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  csv_data.loc[index

next step :It reads the CSV, applies suffixes to subsequent rows based on specific single-cell entries, removes those single-cell rows, and saves the modified dataset.

In [ ]:
import pandas as pd
import os

def apply_suffix_and_clean_data(csv_data):
    # Variable to hold the current suffix to apply
    current_suffix = None

    # Iterate through each row in the DataFrame
    for index, row in csv_data.iterrows():
        # Check if the row has only one non-empty cell and it's in the 'Sample' column
        if row.count() == 1 and pd.notna(row['Sample']):
            current_suffix = row['Sample']  # Update the current suffix
        elif current_suffix and pd.notna(row['Sample']):
            # Apply the current suffix to the 'Sample' column if a suffix is present
            csv_data.at[index, 'Sample'] += f" {current_suffix}"

    # Remove rows where only the 'Sample' column is non-empty
    cleaned_csv_data = csv_data.dropna(thresh=2)  # 'thresh=2' means at least two non-NA values

    return cleaned_csv_data

def process_csv_files(input_directory, output_directory):
    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Iterate through each file in the input directory
    for filename in os.listdir(input_directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(input_directory, filename)
            csv_data = pd.read_csv(file_path)

            # Apply suffixes and clean the data
            modified_data = apply_suffix_and_clean_data(csv_data)

            # Save the modified CSV file to the new directory
            output_file_path = os.path.join(output_directory, filename)
            modified_data.to_csv(output_file_path, index=False)
            print(f"Processed and saved: {filename}")

# Specify the directories
input_directory = r"C:\Users\kandi\Desktop\projects\projects\content\poorna\finaltable"
output_directory = r"C:\Users\kandi\Desktop\projects\projects\content\poorna\tablesuffixed"

# Process all CSV files in the directory
process_csv_files(input_directory, output_directory)



Processed and saved: table_page_3_table_1.png_part_1.csv
Processed and saved: table_page_4_table_1.png_part_1.csv
Processed and saved: table_page_4_table_2.png_part_1.csv
Processed and saved: table_page_5_table_1.png_part_1.csv


In [41]:
!pip install pdfplumber

  Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ----------------------------- ---------- 4.2/5.6 MB 25.2 MB/s eta 0:00:01
   ---------------------------------------- 5.6/5.6 MB 22.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 56.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 3.1/3.1 MB 87.7 MB/s eta 0:00:00
Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl (181 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)


In [ ]:
import pdfplumber
import re

# Function to extract DOI from a PDF and create a link
def extract_doi_link(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                # Regex to find DOI
                match = re.search(r'\b10\.\d{4,9}/[-._;()/:A-Z0-9]+\b', text, re.IGNORECASE)
                if match:
                    # Create a link using the DOI
                    return f"https://doi.org/{match.group(0)}"
    return "DOI not found"

# Example usage
# pdf_path = r"D:\InternshipProject\projects\content\sample_data\1.pdf"
doi_link = extract_doi_link(pdf_path)
print(doi_link)



https://doi.org/10.1016/j.foodchem.2017.07.129


In [ ]:
import pandas as pd
import os
import pdfplumber
import re

# Define amino acid mapping
amino_acid_mapping = {
    # Single letter and full name mappings to three-letter codes
    'A': 'ALA', 'C': 'CYS', 'D': 'ASP', 'E': 'GLU', 'F': 'PHE',
    'G': 'GLY', 'H': 'HIS', 'I': 'ILE', 'K': 'LYS', 'L': 'LEU',
    'M': 'MET', 'N': 'ASN', 'P': 'PRO', 'Q': 'GLN', 'R': 'ARG',
    'S': 'SER', 'T': 'THR', 'V': 'VAL', 'W': 'TRP', 'Y': 'TYR',
    'Alanine': 'ALA', 'Cysteine': 'CYS', 'Aspartic Acid': 'ASP',
    'Glutamic Acid': 'GLU', 'Phenylalanine': 'PHE', 'Glycine': 'GLY',
    'Histidine': 'HIS', 'Isoleucine': 'ILE', 'Lysine': 'LYS', 'Leucine': 'LEU',
    'Methionine': 'MET', 'Asparagine': 'ASN', 'Proline': 'PRO',
    'Glutamine': 'GLN', 'Arginine': 'ARG', 'Serine': 'SER',
    'Threonine': 'THR', 'Valine': 'VAL', 'Tryptophan': 'TRP', 'Tyrosine': 'TYR'
}

required_headers = ['SAMPLE', 'ASP', 'THR', 'SER', 'GLU', 'PRO', 'GLY',
                    'ALA', 'CYS', 'VAL', 'MET', 'ILE', 'LEU', 'TYR', 'PHE', 'HIS',
                    'LYS', 'ARG', 'TRP']

def extract_doi_link(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                # Regex to find DOI
                match = re.search(r'\b10\.\d{4,9}/[-._;()/:A-Z0-9]+\b', text, re.IGNORECASE)
                if match:
                    # Create a link using the DOI
                    return f"https://doi.org/{match.group(0)}"
    return "DOI not found"

def standardize_column_names(cols):
    mapping = {key.upper(): value for key, value in amino_acid_mapping.items()}
    return {col: mapping.get(col, col) for col in cols}

def process_csv_files(directory, doi):
    csv_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]
    data_frames = []

    for file in csv_files:
        df = pd.read_csv(file)
        df.columns = [col.upper() for col in df.columns]
        mapped_names = standardize_column_names(df.columns)
        df.rename(columns=mapped_names, inplace=True)
        df = df[[col for col in df.columns if col in required_headers]]
        data_frames.append((df, sum(df.columns.isin(required_headers))))

    data_frames.sort(key=lambda x: x[1], reverse=True)
    primary_df = data_frames[0][0]

    for df, _ in data_frames[1:]:
        for column in df.columns:
            if column in primary_df.columns and primary_df[column].isnull().all():
                primary_df[column] = df[column]

    if doi:
        primary_df['DOI'] = doi  # Add the DOI as a new column in the DataFrame

    return primary_df.loc[:, required_headers + ['DOI'] if doi else required_headers]

# Example usage
# pdf_path = r"D:\InternshipProject\projects\content\sample_data\1.pdf"
directory_path = r"C:\Users\kandi\Desktop\projects\projects\content\poorna\tablesuffixed"
doi = extract_doi_link(pdf_path)

try:
    final_data = process_csv_files(directory_path, doi)
    final_data.to_excel('C:/Users/kandi/Desktop/projects/projects/content/poorna/tablesuffixed/fdoi.xlsx', index=False)
    print("Data successfully saved to Excel.")
except Exception as e:
    print(f"Error: {e}")


Data successfully saved to Excel.


In [ ]:
import pandas as pd
import os
import pdfplumber
import re

# Define amino acid mapping
amino_acid_mapping = {
    # Single letter and full name mappings to three-letter codes
    'A': 'ALA', 'C': 'CYS', 'D': 'ASP', 'E': 'GLU', 'F': 'PHE',
    'G': 'GLY', 'H': 'HIS', 'I': 'ILE', 'K': 'LYS', 'L': 'LEU',
    'M': 'MET', 'N': 'ASN', 'P': 'PRO', 'Q': 'GLN', 'R': 'ARG',
    'S': 'SER', 'T': 'THR', 'V': 'VAL', 'W': 'TRP', 'Y': 'TYR',
    'Alanine': 'ALA', 'Cysteine': 'CYS', 'Aspartic Acid': 'ASP',
    'Glutamic Acid': 'GLU', 'Phenylalanine': 'PHE', 'Glycine': 'GLY',
    'Histidine': 'HIS', 'Isoleucine': 'ILE', 'Lysine': 'LYS', 'Leucine': 'LEU',
    'Methionine': 'MET', 'Asparagine': 'ASN', 'Proline': 'PRO',
    'Glutamine': 'GLN', 'Arginine': 'ARG', 'Serine': 'SER',
    'Threonine': 'THR', 'Valine': 'VAL', 'Tryptophan': 'TRP', 'Tyrosine': 'TYR'
}

required_headers = ['SAMPLE', 'ASP', 'THR', 'SER', 'GLU', 'PRO', 'GLY',
                    'ALA', 'CYS', 'VAL', 'MET', 'ILE', 'LEU', 'TYR', 'PHE', 'HIS',
                    'LYS', 'ARG', 'TRP']

def extract_doi_link(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                # Regex to find DOI
                match = re.search(r'\b10\.\d{4,9}/[-._;()/:A-Z0-9]+\b', text, re.IGNORECASE)
                if match:
                    # Create a link using the DOI
                    return f"https://doi.org/{match.group(0)}"
    return "DOI not found"

def extract_title(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        first_page = pdf.pages[0]
        text = first_page.extract_text()
        if text:
            # Assume the title is the first line of the text
            lines = text.split('\n')
            for i, line in enumerate(lines):
                if any(name in line for name in ['Matthew G. Nosworthy', 'Adam Franczyk']):  # Names of first authors
                    # Return the line just above the author names as the title
                    title_start = max(i - 3, 0)  # Ensure not to go out of index
                    title_end = i
                    return ' '.join(lines[title_start:title_end]).strip()
    return "Title not found"

def standardize_column_names(cols):
    mapping = {key.upper(): value for key, value in amino_acid_mapping.items()}
    return {col: mapping.get(col, col) for col in cols}

def process_csv_files(directory, doi, title):
    csv_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]
    data_frames = []

    for file in csv_files:
        df = pd.read_csv(file)
        df.columns = [col.upper() for col in df.columns]
        mapped_names = standardize_column_names(df.columns)
        df.rename(columns=mapped_names, inplace=True)
        df = df[[col for col in df.columns if col in required_headers]]
        df['TITLE'] = title  # Add the title as a new column in the DataFrame
        data_frames.append((df, sum(df.columns.isin(required_headers))))

    data_frames.sort(key=lambda x: x[1], reverse=True)
    primary_df = data_frames[0][0]

    for df, _ in data_frames[1:]:
        for column in df.columns:
            if column in primary_df.columns and primary_df[column].isnull().all():
                primary_df[column] = df[column]

    if doi:
        primary_df['DOI'] = doi

    return primary_df.loc[:, required_headers + ['DOI', 'TITLE'] if doi else required_headers + ['TITLE']]

# Example usage
# pdf_path = r"D:\InternshipProject\projects\content\sample_data\1.pdf"
directory_path = r"C:\Users\kandi\Desktop\projects\projects\content\poorna\tablesuffixed"
doi = extract_doi_link(pdf_path)
title = extract_title(pdf_path)

try:
    final_data = process_csv_files(directory_path, doi, title)
    final_data.to_excel('C:/Users/kandi/Desktop/projects/projects/content/poorna/tablesuffixed/fd.xlsx', index=False)
    print("Data successfully saved to Excel.")
    print("Data .")
except Exception as e:
    print(f"Error: {e}")


Data successfully saved to Excel.
Data .
